In [2]:
import speakeasy
import pandas as pd
import numpy as np
import os
import time
import pickle

MALWARE_PATH = "../../../data/pe.dataset/"
X86_PATH = MALWARE_PATH + "PeX86Exe/"
X86_RANSOMWARE = X86_PATH + "ransomware/"
files_200 = [x for x in os.listdir(X86_RANSOMWARE)[200:400]]

Most common error in ransomware folder was this:

```
>>> print(failed_aa["error.traceback"].iloc[0])

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/windows/winemu.py", line 1168, in handle_import_func
    rv = self.api.call_api_func(mod, func, argv, ctx=default_ctx)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/winenv/api/winapi.py", line 77, in call_api_func
    return func(mod, self.emu, argv, ctx)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/winenv/api/usermode/kernel32.py", line 3587, in GetVolumeInformation
    cw = self.get_char_width(ctx)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/winenv/api/api.py", line 343, in get_char_width
    raise ApiEmuError('Failed to get character width from function: %s' % (name))
speakeasy.errors.ApiEmuError: Failed to get character width from function: kernel32.GetVolumeInformation
```

This exception is sort of cosmetic - not because of not implemented API call.

According to code under `speakeasy/winenv/api/api.py`:

```
    def get_char_width(self, ctx):
        """
        Based on the API name, determine the character width
        being used by the function
        """
        name = ctx.get('func_name', '')
        if name.endswith('A'):
            return 1
        elif name.endswith('W'):
            return 2
        raise ApiEmuError('Failed to get character width from function: %s' % (name))
```

So really this is because function name goes without ending, e.g. `GetVolumeInformationA`.

After simple fix:
```
if name.endswith('A'):
            return 1
        elif name.endswith('W'):
            return 2
        else:
            # default to A, if character width is not provided
            return 1
```
.. emulation works:

In [3]:
se = speakeasy.Speakeasy()
module = se.load_module(X86_RANSOMWARE + "916bc6fd4d12c7f53f81d4e3bdf21271ac70a928fab0c9622b61e0a4ac504c9d")
se.run_module(module)

report = se.get_report()
pd.json_normalize([x["apis"] for x in report["entry_points"]][0])

,pc,api_name,args,ret_val
0,0x401c89,kernel32.GetVolumeInformation,"[C:\, 0x0, 0x0, 0x1211fdc, 0x0, 0x0, 0x0, 0x0]",0x1
1,0x401cbe,kernel32.GetVersionEx,[0x1211f4c],0x1
2,0x42947a,kernel32.GetProcAddress,"[0x77000000, CreateFileA]",0xfeee0000
3,0x42947a,kernel32.GetProcAddress,"[0x77000000, CreateFileW]",0xfeee0001
4,0x42947a,kernel32.GetProcAddress,"[0x77000000, WriteFile]",0xfeee0002
...,...,...,...,...
168,0x42947a,kernel32.GetProcAddress,"[0x78c00000, getpeername]",0xfeee009c
169,0x402655,kernel32.VirtualAlloc,"[0x0, 0x400, 0x1000, PAGE_EXECUTE_READWRITE]",0x50000
170,0x402681,kernel32.VirtualAlloc,"[0x0, 0x2ac45, 0x1000, PAGE_EXECUTE_READWRITE]",0x7b000
171,0x42b1de,kernel32.VirtualAlloc,"[0x0, 0x200, 0x3000, PAGE_EXECUTE_READWRITE]",0x51000


# RERUN EMULATIONS AND CHECK ERRORS

In [ ]:
# generated within "dump_reports_ransomware_200_400.py"
with open("reports_ransomware_200_400.pickle", "rb") as fhandle:
    reports = pickle.load(fhandle)

In [ ]:
def parse_report(report):
    aa = pd.json_normalize(report)
    ab = pd.json_normalize(aa["entry_points"].iloc[0])
    try:
        if ab["apis"].any():
            #print(ab["apis"])
            ac = pd.json_normalize(ab["apis"].iloc[0])
        else:
            ac = []
    except AttributeError:
        ac = []
    return aa, ab, ac

bs = []
for i,f in enumerate(files_200):
    a, b, c = parse_report(reports[f])
    if "error.type" in b.keys():
        bs.append(b)
        print(f," : " ,b["error.type"].iloc[0])
    else:
        print(f, " : success")